<h1>Pymaceuticals, Inc. Analysis</h1>
<hr></hr>

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
df_combined = study_results.merge(mouse_metadata, on="Mouse ID")

# Display the data table for preview
df_combined.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [2]:
len(df_combined['Mouse ID'].unique())

249

In [3]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicate_ids = df_combined.loc[df_combined.duplicated(subset=['Mouse ID', 'Timepoint'], keep = False)]["Mouse ID"]
duplicate_ids

860    g989
861    g989
862    g989
863    g989
864    g989
865    g989
866    g989
867    g989
868    g989
869    g989
Name: Mouse ID, dtype: object

In [4]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_rows_df = df_combined.loc[df_combined["Mouse ID"] == 'g989']
duplicate_rows_df

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
860,g989,0,45.000000,0,Propriva,Female,21,26
861,g989,0,45.000000,0,Propriva,Female,21,26
862,g989,5,48.786801,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
864,g989,10,51.745156,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
866,g989,15,51.325852,1,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
868,g989,20,55.326122,1,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
df_cl = df_combined.loc[df_combined["Mouse ID"] != "g989"]
df_cl.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [6]:
# Checking the number of mice in the clean DataFrame.

len(df_cl["Mouse ID"].unique())

248

<h2>Summary Statistics</h2>

In [89]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.

df_per_drug = df_cl.groupby(by = "Drug Regimen")

# Mean
df_per_drug_mean = df_per_drug["Tumor Volume (mm3)"].mean()

# Median
df_per_drug_median = df_per_drug["Tumor Volume (mm3)"].median()

# Variance
df_per_drug_variance = df_per_drug["Tumor Volume (mm3)"].var()

# Standard deviation
df_per_drug_std_var = df_per_drug["Tumor Volume (mm3)"].std()


# Standard error of the mean
df_per_drug_sem = df_per_drug["Tumor Volume (mm3)"].sem()

df_per_drug_stats = pd.concat([df_per_drug_mean, df_per_drug_median, df_per_drug_variance,
                              df_per_drug_std_var,
                              df_per_drug_sem], axis=1, keys=["Mean Tumor Volume",
                                                              "Median Tumor Volume",
                                                              "Tumor Volume Variance",
                                                              "Tumor Volume Std. Dev.",
                                                              "Tumor Volume Std. Err."
                                                             ])

df_per_drug_stats

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std. Dev.,Tumor Volume Std. Err.
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [91]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line

#df_per_drug_agg_stats = df_per_drug.drop(columns=['Mouse ID', 'Sex'])
df_per_drug_agg_stats = df_per_drug.agg(["mean", "median",'var','std','sem'])["Tumor Volume (mm3)"]



/var/folders/8k/v_672r5j2fj3cgw_rm729qrm0000gp/T/ipykernel_14025/4080608291.py:7: FutureWarning: ['Mouse ID', 'Sex'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_per_drug_agg_stats = df_per_drug.agg(["mean", "median",'var','std','sem'])["Tumor Volume (mm3)"]
